<h1>kNN model on kaggle-pet competition</h1>

we are following Stanford CS231n, and trying to apply their assignements to this competition.

CS231N: http://cs231n.github.io/assignments2018/assignment1/
<p>sci-kit knn: https://scikit-learn.org/stable/modules/neighbors.html

Main differences between assignment and this notebook:
    - we will be using Knn for regression task, and not for classification
    - we will be using sci-kit implementation of knn and not written by us
    

In [48]:
import pandas as pd
import numpy as np

In [49]:
from sklearn import neighbors

ModuleNotFoundError: No module named 'sklearn'

In [40]:
import os
os.listdir('../data')
assert 'out_breed.csv' in os.listdir('../data') # this assert breaks if the data is configured uncorrectly

In [15]:
breeds = pd.read_csv('../data/out_breed.csv')
colors = pd.read_csv('../data/out_color.csv')
states = pd.read_csv('../data/out_state.csv')
train  = pd.read_csv('../data/out_train.csv')
test   = pd.read_csv('../data/out_test.csv')
sub    = pd.read_csv('../data/out_submission.csv')

In [43]:
class PredictiveModel(object):
    """
    this is the base class for the prediction task of Adoption Prediction competition
    """
    
    def __init__(self, name):
        self.name = name
        self.model = None
        from time import ctime
        print("{} [{}.__init__] initialized succesfully".format(ctime(), self.name))
        
    def train(self, X, Y):
        """
        train method, feature generation is inside here, data cleaning outside
        
        Args:
            X: pandas.DataFrame, shape = (, 24)
            Y: pandas.Series
        Returns:
            model (also saved in self.model)
        """
        
    def predict(self, X):
        """
        predict method, feature generation is inside here, data cleaning outside
        
        Args:
            X: pandas.DataFrame, shape = (, 24)
        Returns:
            Y: pandas.Series
            
        Raise:
            .not trained
        """

In [45]:
train.shape

(14993, 26)

In [47]:
train.head()

,Unnamed: 0,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,dataset_type
0,0,2,Nibble,3,299,0,1,1,7,0,...,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,train
1,1,2,No Name Yet,1,265,0,1,1,2,0,...,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,train
2,2,1,Brisco,1,307,0,1,2,7,0,...,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,train
3,3,1,Miko,4,307,0,2,1,2,0,...,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,train
4,4,1,Hunter,1,307,0,1,1,0,0,...,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,train
